In [2]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define the preprocessing and data augmentation transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])







In [6]:
# Load the dataset
dataset = ImageFolder(root='/content/drive/MyDrive/design credit/gr 1-2-3 new expt 2023', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [7]:
# Define the model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:


# Instantiate the model and define the loss and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using CUDA


In [11]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.to(device)
        labels = labels.to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

RuntimeError: ignored

In [ ]:
import PIL
from PIL import Image
import numpy as np

# Load the image and preprocess it
img = Image.open('path/to/image')
img = transform(img)
img = np.expand_dims(img, axis=0)
img = torch.from_numpy(img)

# Make a prediction using the trained model
with torch.no_grad():
    outputs = model(img)
    probs = torch.softmax(outputs, dim=1)
    _, predicted = torch.max(probs, 1)

# Print the predicted class label and its corresponding probability
classes = ['non-circular', 'circular']
print(classes[predicted.item()], probs[0][predicted.item()].item())


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc

# Set the model to evaluation mode
model.eval()

# Make predictions on the test dataset
y_true = []
y_pred = []
probs = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true += labels.numpy().tolist()
        y_pred += predicted.numpy().tolist()
        probs += torch.softmax(outputs, dim=1)[:, 1].numpy().tolist()

# Print the classification report and confusion matrix
target_names = ['non-circular', 'circular']
print(classification_report(y_true, y_pred, target_names=target_names))
print(confusion_matrix(y_true, y_pred))

# Plot the ROC curve and calculate the AUC score
fpr, tpr, thresholds = roc_curve(y_true, probs)
roc_auc = roc_auc_score(y_true, probs)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot the precision-recall curve and calculate the AUC score
precision, recall, thresholds = precision_recall_curve(y_true, probs)
pr_auc = auc(recall, precision)
plt.plot(recall, precision, label='PR curve (area = %0.2f)' % pr_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower right")
plt.show()
